# 第二章处理文本数据

## 2.2文本分词

### 载入数据

In [1]:
#读取短篇小说The Verdict作为文本
with open('the-verdict.txt','r',encoding='utf-8') as f:
    raw_text = f.read()
print('total number of characters:',len(raw_text))
print(raw_text[:99])

total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### 分割文本

In [2]:
import re
text = 'hello, world. this is a test'
result = re.split(r'([,.]|\s)',text)#按照逗号句号以及空白处分割
result = [item for item in result if item.strip()]#去掉空白
print(result)

['hello', ',', 'world', '.', 'this', 'is', 'a', 'test']


#### 考虑其他标点

In [3]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item for item in result if item.strip()]##去掉两端的空白字符 也是去掉了空字符串与仅包含空白字符的项
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [4]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]##去掉两端的空白字符 也是去掉了空字符串与仅包含空白字符的项

print(len(preprocessed))


4690


## 2.3 将词元转换为词元id

In [5]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print('vocab size:',vocab_size)

vocab size: 1130


### 创建词汇表

In [6]:
vocab = {word:i for i , word in enumerate(all_words)}
for i , item in enumerate(vocab.items()):
    print(item)
    if i > 10:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)


### 实现简单的文本分类器

In [7]:
class SimpleTokenizerV1:
    def __init__(self,vocab):
        self.str_to_id = vocab
        self.id_to_str = {i:s for s,i in vocab.items()}
    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip()]
        ids = [self.str_to_id[item] for item in preprocessed]
        return ids
    def decode(self,ids):
        text = ' '.join([self.id_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [8]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)
tokenizer.decode(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [9]:
text = "Hello, do you like tea. Is this-- a test?"
tokenizer.encode(text)

KeyError: 'Hello'

## 2.4 引入特殊上下文词元

### 添加特殊词元

In [10]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(['endoftext','<unk>'])
vocab = {token:integer for integer, token in enumerate(all_tokens)}
print(len(vocab))

1132


## 2.5 BPE

In [1]:
import tiktoken

tokenizer = tiktoken.get_encoding('gpt2')

text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text,allowed_special={'<|endoftext|>'})
strings = tokenizer.decode(integers)
integers,strings

([15496,
  11,
  466,
  345,
  588,
  8887,
  30,
  220,
  50256,
  554,
  262,
  4252,
  18250,
  8812,
  2114,
  1659,
  617,
  34680,
  27271,
  13],
 'Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.')

In [2]:
text = 'Akwirw ier'
integers = tokenizer.encode(text)
string = [tokenizer.decode([i]) for i in integers]
strings = tokenizer.decode(integers)
integers,string,strings

([33901, 86, 343, 86, 220, 959],
 ['Ak', 'w', 'ir', 'w', ' ', 'ier'],
 'Akwirw ier')

## 2.6 使用滑动窗口进行数据采样

### 建立用于批处理输入和目标的数据集

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import tiktoken
class GPTDatasetV1(Dataset):
    def __init__(self,text,tokenizer,max_length,stride):
        super().__init__()
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(text)
        for i in range(0,len(token_ids)-max_length,stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx],self.target_ids[idx]


### 建立用于批量生成输入-目标对的数据加载器

In [5]:
def create_dataloader_v1(text,batch_size,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=4):
    tokenizer = tiktoken.get_encoding('gpt2')
    dataset = GPTDatasetV1(text,tokenizer,max_length,stride)
    dataloader = DataLoader(dataset,batch_size=batch_size,shuffle=shuffle,drop_last=drop_last,num_workers=num_workers)
    return dataloader

### 测试数据加载器

In [9]:
with open('the-verdict.txt','r',encoding='utf-8') as f:
    raw_text = f.read()
dataloader = create_dataloader_v1(raw_text,batch_size=8,max_length=4,stride=2,shuffle=False)
data_iter = iter(dataloader)
x,y = next(data_iter)
print(x)
print(y)

tensor([[   40,   367,  2885,  1464],
        [ 2885,  1464,  1807,  3619],
        [ 1807,  3619,   402,   271],
        [  402,   271, 10899,  2138],
        [10899,  2138,   257,  7026],
        [  257,  7026, 15632,   438],
        [15632,   438,  2016,   257],
        [ 2016,   257,   922,  5891]])
tensor([[  367,  2885,  1464,  1807],
        [ 1464,  1807,  3619,   402],
        [ 3619,   402,   271, 10899],
        [  271, 10899,  2138,   257],
        [ 2138,   257,  7026, 15632],
        [ 7026, 15632,   438,  2016],
        [  438,  2016,   257,   922],
        [  257,   922,  5891,  1576]])


## 2.7 创建词元嵌入及编码词元位置信息

In [12]:
input_ids = torch.tensor([2,3,5,1])

vocab_size = 6
out_dim = 3
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, out_dim)
print(embedding_layer.weight)
print(input_ids)
print(embedding_layer(input_ids))

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
tensor([2, 3, 5, 1])
tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [14]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [13]:
max_length = 4
dataloader = create_dataloader_v1(raw_text,batch_size=8,max_length=max_length,stride=max_length,shuffle=False)
data_iter = iter(dataloader)
x,y = next(data_iter)
print(x)
print(x.shape)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
torch.Size([8, 4])


In [15]:
token_embeddings = token_embedding_layer(x)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [16]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

pos_embeddings = pos_embedding_layer(torch.arange(context_length))

print(pos_embeddings.shape)
print(pos_embeddings)


torch.Size([4, 256])
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)


In [17]:
input_embeddings = token_embeddings +pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
